# Feature Engineering
Let's start by importing all of the libraries we will use in this notebook, then let's set up all of our SageMaker variables.

In [68]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import io

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sagemaker import get_execution_role
from sagemaker import PCA

Honestly, using SageMaker for the PCA task we will complete later is overkill for this dataset and sci-kit learn's PCA would suffice in this situation. However, it is fun to use SageMaker and this helps to future proof this code if I extend the project to more geographic regions with many more samples. Maybe a popular real estate company will create a direct MLS API at some point?! A girl can dream. So we will use SageMaker.

In [69]:
# sagemaker libraries
import boto3
import sagemaker
import mxnet as mx

In [38]:
session = sagemaker.Session() # store the current SageMaker session

# get IAM role
role = get_execution_role()
print(role)

arn:aws:iam::359641297910:role/service-role/AmazonSageMaker-ExecutionRole-20190726T141659


In [39]:
# get default bucket
bucket_name = session.default_bucket()
print(bucket_name)
print()

sagemaker-us-east-2-359641297910



In [40]:
# define location to store model artifacts
prefix = 'listings'

output_path='s3://{}/{}/'.format(bucket_name, prefix)

print('Training artifacts will be uploaded to: {}'.format(output_path))

Training artifacts will be uploaded to: s3://sagemaker-us-east-2-359641297910/listings/


In [41]:
active = pd.read_csv('cleaned_active.csv')
past = pd.read_csv('cleaned_past.csv')

In [42]:
print(active.shape)
active.head()

(373, 16)


,SALE TYPE,PROPERTY TYPE,CITY,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE
0,MLS Listing,Single Family Residential,Canyon Country,91351.0,590000.0,4.0,2.0,CAN1 - Canyon Country 1,1577.0,18386.0,1988.0,3.0,374.0,0.0,34.434697,-118.490844
1,MLS Listing,Single Family Residential,Newhall,91321.0,589000.0,3.0,2.5,NEW4 - Newhall 4,2125.0,33397.0,2014.0,34.0,277.0,193.0,34.399271,-118.484278
2,MLS Listing,Single Family Residential,Saugus,91390.0,664900.0,5.0,4.0,BOUQ - Bouquet Canyon,2461.0,6490.0,1979.0,47.0,270.0,0.0,34.461803,-118.495313
3,MLS Listing,Single Family Residential,Canyon Country,91351.0,435000.0,3.0,1.0,CAN1 - Canyon Country 1,920.0,5866.0,1961.0,1.0,473.0,0.0,34.421112,-118.468855
4,MLS Listing,Condo/Co-op,Saugus,91390.0,400000.0,2.0,2.5,COPN - Copper Hill North,1059.0,224705.0,1988.0,1.0,378.0,205.0,34.462818,-118.534228


In [43]:
print(past.shape)
past.head()

(836, 16)


,SALE TYPE,PROPERTY TYPE,CITY,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,LATITUDE,LONGITUDE
0,PAST SALE,Single Family Residential,Canyon Country,91387.0,570000,3.0,2.50,CAN2 - Canyon Country 2,1464.0,10789.0,1983.0,5,389.0,0.0,34.449485,-118.400176
1,PAST SALE,Single Family Residential,Canyon Country,91387.0,535000,3.0,2.00,CAN2 - Canyon Country 2,1548.0,24285.0,1974.0,69,346.0,0.0,34.457457,-118.389404
2,PAST SALE,Single Family Residential,Canyon Country,91387.0,525000,4.0,2.00,CAN2 - Canyon Country 2,1326.0,7336.0,1975.0,37,396.0,0.0,34.454962,-118.391801
3,PAST SALE,Single Family Residential,Canyon Country,91387.0,560000,4.0,2.00,CAN2 - Canyon Country 2,1854.0,6628.0,1991.0,31,302.0,0.0,34.440028,-118.388719
4,PAST SALE,Single Family Residential,Canyon Country,91387.0,560000,3.0,1.75,CAN2 - Canyon Country 2,1479.0,8662.0,1965.0,18,379.0,45.0,34.427945,-118.425415


## Prepare Data
Now that we have clean our data, let's prep it for our model. We need to extract the price feature to use for our target vector (y). We also want to convert all of our categorical variables to numerical values; we will use one hot encoding to accomplish this task.

In [44]:
# Concatenate active and past samples temporarily in order to keep a consistent feature set after one hot encoding
combined = pd.concat([active, past])
combined.shape

(1209, 16)

In [45]:
# Convert categorical data to numerical
cat_cols = ['SALE TYPE', 'PROPERTY TYPE', 'CITY', 'LOCATION']
combined = pd.get_dummies(combined, columns=cat_cols)
combined.shape

(1209, 107)

In [46]:
active, past = combined[:len(active)], combined[len(active):]
print("Shape of Active Samples: ", active.shape)
print("Shape of Past Samples: ", past.shape)

Shape of Active Samples:  (373, 107)
Shape of Past Samples:  (836, 107)


In [47]:
# Create target vectors
y_active = active['PRICE']
y_past = past['PRICE']

# drop price column from X features
active = active.drop('PRICE', axis=1)
past = past.drop('PRICE', axis=1)

In [48]:
active.head()

,ZIP OR POSTAL CODE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,LATITUDE,...,LOCATION_VVER - Val Verde,LOCATION_VWES - Valencia Westridge,LOCATION_Valencia - Avanti at Westcreek,LOCATION_Valencia 1,LOCATION_Valencia Creekside,LOCATION_Valencia North,LOCATION_Valencia Northbridge,LOCATION_Valencia West Creek,LOCATION_Valencia Westridge,LOCATION_Verano at Aliento Active Adult
0,91351.0,4.0,2.0,1577.0,18386.0,1988.0,3.0,374.0,0.0,34.434697,...,0,0,0,0,0,0,0,0,0,0
1,91321.0,3.0,2.5,2125.0,33397.0,2014.0,34.0,277.0,193.0,34.399271,...,0,0,0,0,0,0,0,0,0,0
2,91390.0,5.0,4.0,2461.0,6490.0,1979.0,47.0,270.0,0.0,34.461803,...,0,0,0,0,0,0,0,0,0,0
3,91351.0,3.0,1.0,920.0,5866.0,1961.0,1.0,473.0,0.0,34.421112,...,0,0,0,0,0,0,0,0,0,0
4,91390.0,2.0,2.5,1059.0,224705.0,1988.0,1.0,378.0,205.0,34.462818,...,0,0,0,0,0,0,0,0,0,0


In [49]:
past.head()

,ZIP OR POSTAL CODE,BEDS,BATHS,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,LATITUDE,...,LOCATION_VVER - Val Verde,LOCATION_VWES - Valencia Westridge,LOCATION_Valencia - Avanti at Westcreek,LOCATION_Valencia 1,LOCATION_Valencia Creekside,LOCATION_Valencia North,LOCATION_Valencia Northbridge,LOCATION_Valencia West Creek,LOCATION_Valencia Westridge,LOCATION_Verano at Aliento Active Adult
0,91387.0,3.0,2.50,1464.0,10789.0,1983.0,5.0,389.0,0.0,34.449485,...,0,0,0,0,0,0,0,0,0,0
1,91387.0,3.0,2.00,1548.0,24285.0,1974.0,69.0,346.0,0.0,34.457457,...,0,0,0,0,0,0,0,0,0,0
2,91387.0,4.0,2.00,1326.0,7336.0,1975.0,37.0,396.0,0.0,34.454962,...,0,0,0,0,0,0,0,0,0,0
3,91387.0,4.0,2.00,1854.0,6628.0,1991.0,31.0,302.0,0.0,34.440028,...,0,0,0,0,0,0,0,0,0,0
4,91387.0,3.0,1.75,1479.0,8662.0,1965.0,18.0,379.0,45.0,34.427945,...,0,0,0,0,0,0,0,0,0,0


## Standardize Data + Dimensionality Reduction
Next we will standardize our data and then use Principle Component Analysis (PCA) to help reduce the number of features used in our model since the current ratio of degrees of freedom (DF) to samples is very high, which has a tendancy to lead to overfitting models during training.

In [56]:
# Standardize features
X_active = StandardScaler().fit_transform(active.astype(float))
X_past = StandardScaler().fit_transform(past.astype(float))

In [57]:
print(X_active)

[[-0.45269146  0.8786265  -0.71067211 ... -0.07342231  0.
  -0.05184758]
 [-1.96121362 -0.1328787   0.12952027 ... -0.07342231  0.
  -0.05184758]
 [ 1.50838735  1.8901317   2.65009743 ... -0.07342231  0.
  -0.05184758]
 ...
 [-0.30183924 -0.1328787   0.12952027 ... -0.07342231  0.
  -0.05184758]
 [-0.30183924 -0.1328787   0.12952027 ... -0.07342231  0.
  -0.05184758]
 [-0.30183924 -0.1328787   0.12952027 ... -0.07342231  0.
  -0.05184758]]


In [58]:
print(X_past)

[[ 1.41862331 -0.18297938  0.17453775 ... -0.03460643 -0.03460643
   0.        ]
 [ 1.41862331 -0.18297938 -0.69399535 ... -0.03460643 -0.03460643
   0.        ]
 [ 1.41862331  0.93359553 -0.69399535 ... -0.03460643 -0.03460643
   0.        ]
 ...
 [-0.22399315 -0.18297938  0.17453775 ... -0.03460643 -0.03460643
   0.        ]
 [-0.22399315 -0.18297938  0.17453775 ... -0.03460643 -0.03460643
   0.        ]
 [-0.22399315 -0.18297938  1.04307086 ... -0.03460643 -0.03460643
   0.        ]]


In [74]:
# Define PCA model
NUM_COMPONENTS = len(X_past[0]) -1
print("Number of initial components: ", NUM_COMPONENTS)

pca = PCA(role=role,
          train_instance_count=1,
          train_instance_type='ml.c4.xlarge',
          output_path=output_path,
          num_components=NUM_COMPONENTS,
          sagemaker_session=session)

Number of initial components:  105


In [65]:
# Convert X_past to RecordSet format for compatibility with SageMaker's PCA model
record_set_X_past = pca.record_set(X_past)

In [66]:
%%time

# Train PCA model on the now properly formatted X_past matrix
pca.fit(record_set_X_past)

2019-09-04 02:17:13 Starting - Starting the training job...
2019-09-04 02:17:15 Starting - Launching requested ML instances......
2019-09-04 02:18:15 Starting - Preparing the instances for training......
2019-09-04 02:19:22 Downloading - Downloading input data...
2019-09-04 02:20:11 Training - Training image download completed. Training in progress.
2019-09-04 02:20:11 Uploading - Uploading generated training model
Docker entrypoint called with argument(s): train
[09/04/2019 02:20:02 INFO 139949399766848] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'_num_gpus': u'auto', u'_log_level': u'info', u'subtract_mean': u'true', u'force_dense': u'true', u'epochs': 1, u'algorithm_mode': u'regular', u'extra_components': u'-1', u'_kvstore': u'dist_sync', u'_num_kv_servers': u'auto'}
[09/04/2019 02:20:02 INFO 139949399766848] Reading provided configuration from /opt/ml/input/config/hyperparameters.json: {u'feature_dim': u'106',


2019-09-04 02:20:16 Completed - Training job completed
Training seconds: 54
Billable seconds: 54
CPU times: user 416 ms, sys: 22.8 ms, total: 439 ms
Wall time: 3min 41s


## Analyze PCA Model Attributes

In [70]:
# Training job name is simply copied from the AWS console
training_job_name = 'pca-2019-09-04-02-17-12-955'

# Declare where AWS is saving the model by default
model_key = os.path.join(prefix, training_job_name, 'output/model.tar.gz')
print(model_key)

# Download and unzip model
boto3.resource('s3').Bucket(bucket_name).download_file(model_key, 'model.tar.gz')

# unzipping as model_algo-1
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

listings/pca-2019-09-04-02-17-12-955/output/model.tar.gz


2304

In [71]:
# Load the model's artifacts
pca_params = mx.ndarray.load('model_algo-1')
print(pca_params)

{'s': 
[          nan           nan           nan           nan           nan
           nan           nan           nan           nan           nan
           nan           nan           nan           nan           nan
           nan 0.0000000e+00 1.1981063e-15 4.3674848e-15 1.7133534e-07
 1.8462460e-07 2.0969685e-07 2.6340967e-07 3.6333847e-07 3.8010950e-07
 4.8563481e-07 5.3896616e-07 6.2952404e-07 6.9387210e-07 7.9633207e-07
 8.2101502e-07 8.7104784e-03 1.7413080e-02 2.2847727e-02 2.7881355e+00
 3.8385134e+00 6.3892760e+00 7.5989285e+00 8.3918486e+00 9.7023506e+00
 1.2050980e+01 1.4972833e+01 1.5480830e+01 2.2774343e+01 2.3518904e+01
 2.5282520e+01 2.6153364e+01 2.8172110e+01 2.8931345e+01 2.8932592e+01
 2.8933537e+01 2.8934616e+01 2.8938049e+01 2.8939375e+01 2.8940874e+01
 2.8944010e+01 2.8948643e+01 2.8950512e+01 2.8951149e+01 2.8954199e+01
 2.8956192e+01 2.8964170e+01 2.8969248e+01 2.8973907e+01 2.8977901e+01
 2.8978565e+01 2.8987282e+01 2.8999123e+01 2.9012304e+01 2.9020258e+01

In [72]:
# Save s = percentage of variance from the projected feature space
s = pd.DataFrame(pca_params['s'].asnumpy())

# Save v = makeup of the principal components
v = pd.DataFrame(pca_params['v'].asnumpy())

In [82]:
# Number of highest variance principal components (PCs) to investigate
num_PCs = 20

# Starting index, since highest variance components are located at the end of the rows
start_here = NUM_COMPONENTS - num_PCs

print(s.iloc[start_here:, :])

             0
85   29.505402
86   29.664145
87   29.751223
88   29.863804
89   30.236134
90   30.673931
91   31.495037
92   32.178860
93   32.441799
94   33.346672
95   35.084782
96   40.619595
97   41.031532
98   42.181431
99   43.512646
100  43.947289
101  48.523144
102  49.437016
103  53.205795
104  64.985840


In [83]:
def explained_variance(s, n_top_components):
    '''Calculates the approx. data variance that n_top_components captures.
       :param s: A dataframe of singular values for top components; 
           the top value is in the last row.
       :param n_top_components: An integer, the number of top components to use.
       :return: The expected data variance covered by the n_top_components.'''
    
    start_here = NUM_COMPONENTS - n_top_components
    
    # calculate approx variance
    exp_variance = np.square(s.iloc[start_here:,:]).sum()/np.square(s).sum()
    
    return exp_variance[0]

In [107]:
# test cell
n_top_components = 63 # select a value for the number of top components

# calculate the explained variance
exp_variance = explained_variance(s, n_top_components)
print('Explained variance: ', exp_variance)

Explained variance:  0.99020576


## Deploy the PCA Model
Now that we have found the number of Principal Components that explains our desired amount of total variance, we can deploy our model for training.

In [ ]:
%%time

pca_predictor = pca.deploy(initial_instance_count=1, 
                           instance_type='ml.t2.medium')

In [ ]:
# Train PCA model with numpy data matrix
train_pca = pca_predictor.predict(X_past)

# Confirm train_pca has the expected format of training features
print(train_pca[0])